In [105]:
import numpy as np
import random
import pygame
import copy



In [106]:
class environement:
    
    def __init__(self):
        self.state=None
        self.width=None
        self.height=None
        self.snake_list=[]
        self.foodx=None
        self.foody=None
        self.distance=None
    def env_init(self,info={}):
        self.width=info.get("width",10)
        self.height=info.get("height",10)
        self.state=np.zeros((self.width,self.height))
    def spawnfood(self):
        foodx=random.randint(0,self.width-1)
        foody=random.randint(0,self.height-1)
        for x in self.snake_list:
            if(foodx==x[0] and foody==x[1]):
                foodx,foody=self.spawnfood()
        return foodx,foody
    def Distance(self,snake_head,foodx,foody):
        return (abs((snake_head[0]-foodx))+abs((snake_head[1]-foody)))
    def arrange_state(self):
        self.state=np.zeros((self.width,self.height))
        for x,y ,in self.snake_list:
            if(x==self.snake_list[-1][0] and y==self.snake_list[-1][1]):
                self.state[x][y]=2
            else:
                self.state[x][y]=1
        self.state[self.foodx][self.foody]=5

        
    def start(self):
        self.snake_head=[int(self.width/2),int(self.height/2)]
        self.snake_list=[]
        a=[]
        a.append(self.snake_head[0])
        a.append(self.snake_head[1])
        self.snake_list.append(a)
        self.foodx,self.foody=self.spawnfood()
        self.arrange_state()
        self.distance=self.Distance(self.snake_head,self.foodx,self.foody)
        return self.state
    def rew(self,reward):
            if(reward=="food"):
                return 1
            elif(reward=="death"):
                return -1
            elif(reward=="moveclose"):
                return 0
            elif(reward=="movefar"):
                return -0.05
    def step(self,action):
        #action 0:L 1:R 2:D 3:U:
        return self.step_arrange(action)
    def step_arrange(self,action):
        if action==0:
            index=1
            incr=-1
            side=self.height
        elif action==1:
            index=1
            incr=1
            side=self.height
        elif action==2:
            index=0
            incr=-1
            side=self.width
        else :        
            index=0
            incr=1
            side=self.width
        self.snake_head[index]+=incr
        a=[]
        a.append(self.snake_head[0])
        a.append(self.snake_head[1])
        if(self.snake_head[0]==self.foodx and self.snake_head[1]==self.foody):
            self.snake_list.append(a)
            self.foodx,self.foody=self.spawnfood()
            self.arrange_state()
            return self.rew("food"),self.state,False
        if self.snake_head[index]<0 or self.snake_head[index]>=side:
            self.start()
            return self.rew("death"),self.state,True
        for x in self.snake_list[1:]:
            if x == self.snake_head:
                self.start()
                return self.rew("death"),self.state,True
        del self.snake_list[0]
        self.snake_list.append(a)
        self.arrange_state()
        new_distance=self.Distance(self.snake_head,self.foodx,self.foody)
        if new_distance>=self.distance :
            self.distance=new_distance
            return self.rew("movefar"),self.state,False
        else:
            self.distance=new_distance
            return self.rew("moveclose"),self.state,False
        
        
        
        
        
        
   
        
        